# 调用get_kw_
参数为：ref_file  为参考文献的文件
cooccurrence_csv   要将提取的关键知识保存到哪
processed_csv   要将已处理的参考文献保存到哪

In [3]:
import nltk
#nltk.download('punkt_tab')
#nltk.download('punkt')
from collections import Counter
import re
from itertools import combinations
from keybert import KeyLLM
import openai
from keybert.llm import OpenAI
def extract_keywords(abstract):
    prompt1 = """
    I have the following abstract:
    [DOCUMENT]
    
    Based on the Abstract above, extract the keywords that best describe the topic of the abstract.
    Make sure all extracted keywords or concepts appear verbatim in the text and do not exceed ten keywords.
    Use the following format separated by commas:
    <keywords>
    """
    prompt2 = """
    **Instructions**:
    - You are given an academic abstract.
    - Your task is to extract the most important pieces of knowledge or concepts that:
        - Describe the core topic of the paper.
        - Represent the key knowledge central to the study.
        - Are directly used in the paper’s research method, framework, or experimental design.
        - Reflect the new theories, methods, or contributions introduced by the paper.
        - Include key theories, frameworks, or technologies used.
        - Represent the scientific or practical contribution to the field.
    - All extracted items must appear verbatim in the abstract.
    - Do not paraphrase, summarize, or add inferred content.
    - Try to extract no more than ten distinct items.
    - Return the final result as a single line, separated by English commas.
    **Now process the following abstract:**
    I have the following abstract: 
    [DOCUMENT]
    Based on the abstract above, extract the knowledge that best describes the topic of the abstract.
    Prioritize any concepts that relate to the paper’s innovation, methodology, and research contribution.
    Make sure all extracted knowledge or concepts appear verbatim in the text and try not to exceed ten pieces.
    Use the following format separated by commas:
    <keywords>
    """
    prompt = prompt1

    base_url = 
    api_key = 
    # Create your LLM
    client  = openai.OpenAI(base_url='https://open.api.gu28.top/v1',
        # required but ignored
        api_key=api_key)
    
    #model="llama3.3:70b-instruct-q2_k"  gpt-4o
    llm = OpenAI(client,model="gpt-4o",prompt=prompt,chat=True)
    # Load it in KeyLLM
    kw_model = KeyLLM(llm)
    keywords = kw_model.extract_keywords(abstract,check_vocab=True)
    kws = keywords[0]
    print(kws)
    return kws

def analyze_cooccurrence_standardized(sentences, keywords):
    co_occurrence_counter = Counter()
    num_sentences = len(sentences)
    
    # 预编译正则表达式模式
    keyword_patterns = {kw: re.compile(r'\b' + re.escape(kw.lower()) + r'\b') for kw in keywords}
    
    for i in range(num_sentences):
        # 确定窗口范围
        if i == 0:
            window_sentences = sentences[i:i + 2]  # 第一个句子，包含第1和第2句
        elif i == num_sentences - 1:
            window_sentences = sentences[i - 1:i + 1]  # 最后一个句子，包含倒数第2和第5句
        else:
            window_sentences = sentences[i - 1:i + 2]  # 中间句子，包含前一句、当前句、后一句
        
        window_text = ' '.join(window_sentences).lower()
        
        # 找出窗口中出现的关键词
        present_keywords = [kw for kw in keywords if keyword_patterns[kw].search(window_text)]
        
        # 统计所有可能的关键词对
        for pair in combinations(present_keywords, 2):
            sorted_pair = tuple(sorted(pair))
            co_occurrence_counter[sorted_pair] = 1
    
    return co_occurrence_counter

def liucheng(abstract):
    keyword_list = extract_keywords(abstract)
    sentences = nltk.sent_tokenize(abstract)
    rst = analyze_cooccurrence_standardized(sentences,keyword_list)
    return rst

In [4]:
abstract = """
Large neural networks can now generate jokes, but do they really "understand" humor? We challenge AI models with three tasks derived from the New Yorker Cartoon Caption Contest: matching a joke to a cartoon, identifying a winning caption, and explaining why a winning caption is funny. These tasks encapsulate progressively more sophisticated aspects of "understanding" a cartoon; key elements are the complex, often surprising relationships between images and captions and the frequent inclusion of indirect and playful allusions to human experience and culture. We investigate both multimodal and language-only models: the former are challenged with the cartoon images directly, while the latter are given multifaceted descriptions of the visual scene to simulate human-level visual understanding. We find that both types of models struggle at all three tasks. For example, our best multimodal models fall 30 accuracy points behind human performance on the matching task, and, even when provided ground-truth visual scene descriptors, human-authored explanations are preferred head-to-head over the best machine-authored ones (few-shot GPT-4) in more than 2/3 of cases. We release models, code, leaderboard, and corpus, which includes newly-gathered annotations describing the image's locations/entities, what's unusual in the scene, and an explanation of the joke.
"""
rst = extract_keywords(abstract)
print(rst)

['Large neural networks', 'jokes', 'humor', 'AI models', 'New Yorker Cartoon Caption Contest', 'understanding', 'cartoon', 'visual understanding', 'multimodal models', 'language-only models']
['Large neural networks', 'jokes', 'humor', 'AI models', 'New Yorker Cartoon Caption Contest', 'understanding', 'cartoon', 'visual understanding', 'multimodal models', 'language-only models']


In [4]:
# 加载已有的共现关系
def load_cooccurrence(csv_file):
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file)
        co_occurrence_counter = Counter()
        #print(csv_file)
        for _, row in df.iterrows():
            pair = tuple(sorted([row['Keyword1'], row['Keyword2']]))
            co_occurrence_counter[pair] = row['Count']
        print(f"已加载共现关系，共有 {len(co_occurrence_counter)} 个关键词对。")
    else:
        co_occurrence_counter = Counter()
        print("未找到共现关系文件，初始化一个新的计数器。")
    return co_occurrence_counter

def save_cooccurrence(co_occurrence_counter, csv_file):
    data = {
        'Keyword1': [pair[0] for pair in co_occurrence_counter.keys()],
        'Keyword2': [pair[1] for pair in co_occurrence_counter.keys()],
        'Count': [count for count in co_occurrence_counter.values()]
    }
    df = pd.DataFrame(data)
    df.to_csv(csv_file, index=False)
    print(f"共现关系已保存到 '{csv_file}'。")
def load_processed_titles(processed_csv):
    """
    从 CSV 文件加载已处理摘要的标题。

    Args:
        processed_csv (str): 存储已处理标题的 CSV 文件路径。

    Returns:
        set: 已处理标题的集合。
    """
    if os.path.exists(processed_csv):
        processed_df = pd.read_csv(processed_csv)
        processed_titles = set(processed_df['Title'].tolist())
        print(f"已加载 {len(processed_titles)} 个已处理的摘要标题。")
    else:
        processed_titles = set()
        print("未找到已处理摘要记录文件，初始化一个新的记录。")
    return processed_titles
def save_processed_titles(new_titles, processed_csv):
    """
    将新增处理的标题保存到 CSV 文件。

    Args:
        new_titles (list): 新处理的标题列表。
        processed_csv (str): 存储已处理标题的 CSV 文件路径。
    """
    if os.path.exists(processed_csv):
        processed_df = pd.read_csv(processed_csv)
    else:
        processed_df = pd.DataFrame(columns=['Title'])
    
    # 合并新处理的标题
    new_df = pd.DataFrame({'Title': new_titles})
    processed_df = pd.concat([processed_df, new_df], ignore_index=True)
    processed_df.to_csv(processed_csv, index=False)
    print(f"已更新 {len(new_titles)} 个已处理的摘要标题。")


In [5]:
import pandas as pd
import os
def get_kw_(ref_file,cooccurrence_csv,processed_csv):
    # 加载已处理标题
    processed_titles = load_processed_titles(processed_csv)
    # 加载CSV文件
    df = pd.read_csv(ref_file)
    titles = df['title'].dropna().tolist()
    abstracts = df['abstract'].dropna().tolist()
    # 加载共现关系
    co_occurrence_counter = load_cooccurrence(cooccurrence_csv)
    # 新处理的标题
    new_processed_titles = []
    for title, abstract in zip(titles, abstracts):
        if title in processed_titles:
            continue  # 跳过已处理的摘要
        #print(f'处理到{title}')
        # 睡眠3秒
        time.sleep(1)
        #print(abstract)
        co_occurrence_counter1 = liucheng(abstract)
        if co_occurrence_counter == None:
            co_occurrence_counter = co_occurrence_counter1
        else:
            co_occurrence_counter += co_occurrence_counter1
        # 记录新处理的标题
        new_processed_titles.append(title)
    # 更新已处理的标题记录
    if new_processed_titles:
        save_processed_titles(new_processed_titles, processed_csv)
    # 保存共现关系到 CSV 文件
    save_cooccurrence(co_occurrence_counter, cooccurrence_csv)
    print("处理完成。")
    

## 获得所有的共现关系

In [16]:
# get_kw_(ref_file='keybert/ref.csv', cooccurrence_csv = 'keybert/3-r-coocurrence.csv',processed_csv= 'keybert/processed_csv')

# 读取all_focus.xlsx
import pandas as pd
import time
df = pd.read_excel('data1/all_A_papers.xlsx')

if 'co_occurrence_counter' not in df.columns:
    df['co_occurrence_counter'] = None
    
# 得到Title
titles = df['title'].tolist()
# 我想根据df的索引来，因为我不但要读titles还有读取abstract列
#abstracts = df['Abstract'].tolist()
# for循环，得到index title和abstract
has_done = 1473
for index,title in enumerate(titles):
    if index <= has_done:
        continue
    time.sleep(1)
    try:
        get_kw_(ref_file=f'data1/{index}_ref.csv', cooccurrence_csv = f'data1/{index}_cooccurrence.csv',processed_csv= f'data1/{index}_processed.csv')
    except FileNotFoundError as e:
        continue
    # 判断索引为index，的co_occurrence_counter是否不为空，如果不为空就continue
    if pd.notna(df.loc[index,'co_occurrence_counter']):
        continue
    else:
        # 读取{index}_paper.csv文件，
        df1 = pd.read_csv(f'data1/{index}_paper.csv')
        # 得到abstract列的第一行
        abstract = df1['abstract'][0]
        # 判断abstract是否为空，或者为nan，如果是就continue
        if pd.isna(abstract) or not abstract:
            continue
        # 设置2s休眠
        co_occurrence_counter = liucheng(abstract)
        # 在all_focus.xlsx中加入新，为co_occurrence_counter，然后报错格式为Counter的co_occurrence_counter
        df.loc[index,'co_occurrence_counter'] = str(co_occurrence_counter)
        df.to_excel('data1/all_A_papers.xlsx',index=False)
"""
# 需要读取解析的时候使用eval函数
from collections import Counter
df = pd.read_excel('data/all_focus.xlsx')
# 确保所有值都是字符串（跳过 NaN 或非字符串）
def safe_eval(cell):
    if isinstance(cell, str):
        return eval(cell, {'Counter': Counter})
    return cell
# 使用 eval() 解析字符串恢复为 Counter
df['co_occurrence_counter'] = df['co_occurrence_counter'].apply(safe_eval)
# 打印解析后的结果
print(df['co_occurrence_counter'][0])  # 输出: Counter({'This': 1, 'is': 1, 'an': 1, 'example': 1, 'abstract.': 1})
"""

未找到已处理摘要记录文件，初始化一个新的记录。
未找到共现关系文件，初始化一个新的计数器。
['differential privacy', 'empirical privacy estimate', 'private machine learning', 'auditing mechanisms', 'adversarial dataset', 'model updates', 'federated learning', 'tight composition theorems', 'privacy leakage', 'implementation bugs']
['differentially private', 'privacy loss', 'privacy auditing', 'federated settings', 'Gaussian mechanism', 'FL benchmark datasets', 'adversarial threat models']
['machine learning models', 'personal data ownership', 'intellectual property rights', 'statistical techniques', 'training data', 'adversarial tools', 'legal settings', 'low false-positive', 'offline scenarios']
['privacy guarantee', 'differentially private', 'machine learner', 'privacy loss', 'poisoning attacks', 'membership inference', 'privacy violations', 'implementation errors']
['Federated Learning', 'model updates', 'attacks', 'private information', 'Differential Privacy', 'noise', 'threat model', 'CANIFE', 'vision models', 'language models

"\n# 需要读取解析的时候使用eval函数\nfrom collections import Counter\ndf = pd.read_excel('data/all_focus.xlsx')\n# 确保所有值都是字符串（跳过 NaN 或非字符串）\ndef safe_eval(cell):\n    if isinstance(cell, str):\n        return eval(cell, {'Counter': Counter})\n    return cell\n# 使用 eval() 解析字符串恢复为 Counter\ndf['co_occurrence_counter'] = df['co_occurrence_counter'].apply(safe_eval)\n# 打印解析后的结果\nprint(df['co_occurrence_counter'][0])  # 输出: Counter({'This': 1, 'is': 1, 'an': 1, 'example': 1, 'abstract.': 1})\n"

处理遗漏的单篇论文方法

In [22]:
df = pd.read_excel('data1/all_A_papers.xlsx')
abstract = """
Although image restoration has achieved significant progress, its potential to assist object detectors in adverse imaging conditions lacks enough attention. It is reported that the existing image restoration methods cannot improve the object detector performance and sometimes even reduce the detection performance. To address the issue, we propose a targeted adversarial attack in the restoration procedure to boost object detection performance after restoration. Specifically, we present an ADAM-like adversarial attack to generate pseudo ground truth for restoration training. Resultant restored images are close to original sharp images, and at the same time, lead to better results of object detection. We conduct extensive experiments in image dehazing and low light enhancement and show the superiority of our method over conventional training and other domain adaptation and multi-task methods. The proposed pipeline can be applied to all restoration methods and detectors in both one-and two-stage.
"""
co_occurrence_counter = liucheng(abstract)
df.loc[1445,'co_occurrence_counter'] = str(co_occurrence_counter)
df.to_excel('data1/all_A_papers.xlsx',index=False)

['image restoration', 'object detectors', 'adversarial attack', 'restoration procedure', 'ADAM-like', 'pseudo ground truth', 'image dehazing', 'low light enhancement', 'domain adaptation', 'multi-task methods']


In [11]:
need_process = [16, 35, 40, 50, 53, 56, 66, 88, 133, 149, 184, 254, 260, 279, 366, 631, 751, 785, 890, 918, 1082, 1118, 1123, 1217, 1235, 1277, 1324]

In [12]:
import pandas as pd
df = pd.read_excel('data1/all_A_papers.xlsx')
import time
# 找到df中所有co_occurrence_counter为空的，如果为空就进行处理
for index in need_process:
    print(index)
    try:
        get_kw_(ref_file=f'data1/{index}_ref.csv', cooccurrence_csv = f'data1/{index}_cooccurrence.csv',processed_csv= f'data1/{index}_processed.csv')
    except FileNotFoundError as e:
        continue
    if pd.notna(df.loc[index,'co_occurrence_counter']):
        continue
    else:
        df1 = pd.read_csv(f'data1/{index}_paper.csv')
        abstract = df1['abstract'][0]
        if pd.isna(abstract) or not abstract:
            continue
        co_occurrence_counter = liucheng(abstract)
        df.loc[index,'co_occurrence_counter'] = str(co_occurrence_counter)
        df.to_excel('data1/all_A_papers.xlsx',index=False)

751
未找到已处理摘要记录文件，初始化一个新的记录。
未找到共现关系文件，初始化一个新的计数器。
['model evaluation', 'data-oriented', 'representation-oriented', 'process-oriented evaluation', 'rule induction', 'error estimation']
['average case analysis', 'induction algorithms', 'monotone M of N', 'Boolean attributes', 'class noise', 'computational tractability', 'normal distribution', 'supervised induction', 'conditional probability distribution']
['representation-oriented', 'process-oriented evaluation', 'generalization error', 'search process', 'rule induction', 'CN2 rule learner', 'high-dimensional domains', 'simpler models']
['Model selection', 'Bayesian', 'priors on error values', 'expected true error', 'classification learning', 'empirical minimizer', 'error distribution', 'PAC theory', 'VC-dimension', '10-fold cross-validation']
['machine learning algorithms', 'benchmark data sets', 'domain-specific parameters', 'error rate', 'optimistic bias', 'more parameters', 'fewer parameters', 'C4.5', 'FOIL', 'unbiased ranking experi